In [ ]:
# https://arxiv.org/pdf/2104.12259.pdf

In [2]:
import torch
vers = torch.__version__
print("Torch vers: ", vers)

# PyG installation
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

import torch_geometric  # handling graph data

Torch vers:  2.0.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
# optuna library used for the hyperparameter optimization
# different samplers like grid search , random search ,evolutonary algo are used to find the optimal parameters.
# Grid Search: searches in the predetermined subset of whole hyperparameter space of target algorithm.
# Bayesian Search : Used probability distribution to select a value of each hyper-parameter
# Random Search: Randomly search the space untill the stopping criteria are met 
# Evolutionary Algo: fitness function used to find the value of the hyperparameter
# optuna has to componenets : study and trial
# single execution of optimization function called trial
# collection of trials is called study

In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.1 MB/s eta 0:00:00


In [ ]:
# Dataloader iterable object provides interface for batching and iterating over the object.(mini-batches)
# during the testing the shufffling is necesary.

In [4]:
import torch
from torch_geometric.datasets import UPFD
import numpy as np
from torch_geometric.loader import DataLoader

In [5]:
import torch.nn.functional as F
from torch.nn import ReLU, LeakyReLU, Softmax, Linear, SELU, GELU, ELU
from torch_geometric.nn import SAGEConv, global_max_pool, TopKPooling,GCNConv

In [6]:
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score 

In [7]:
test_data_gos = UPFD(root=".", name="gossipcop", feature="bert",split="test")
train_data_gos = UPFD(root=".", name="gossipcop", feature="bert", split="train")
val_data_gos = UPFD(root=".", name="gossipcop", feature="bert", split="val")

Extracting ./gossipcop/raw/uc
Processing...
Done!


In [8]:
train_loader = DataLoader(train_data_gos, batch_size=32, shuffle=True) # shuffle reduces the bias the due to order
test_loader = DataLoader(test_data_gos, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data_gos, batch_size=32, shuffle=True)

In [9]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.leaky1 = LeakyReLU()
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.leaky2 = LeakyReLU()
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        self.leaky3 = LeakyReLU() 
        
        self.full1 = Linear(hidden_channels[2],hidden_channels[3])
        self.ge1 = GELU()
        self.full2 = Linear(hidden_channels[3],hidden_channels[4])
        self.ge2 = GELU()
        self.full3 = Linear(hidden_channels[4],out_channels)


    def forward(self, x, edge_index, batch):
        h = self.leaky1(self.conv1(x, edge_index))
        h = self.leaky2(self.conv2(h, edge_index))
        h = self.leaky3(self.conv3(h, edge_index))

        h = global_max_pool(h,batch)

        h = self.ge1(self.full1(h))
        h = self.ge2(self.full2(h))
        h = self.full3(h)

        return torch.sigmoid(h)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
def objective(trial):
  
  model = Net(768,[trial.suggest_int(name="layer_size1",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size2",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size3",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size4",low=64,high=256,step=64),
                   trial.suggest_int(name="layer_size5",low=64,high=256,step=64),
  
  ],1).to(device)
  optimizer = torch.optim.Adam(model.parameters(),lr=trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),betas=(trial.suggest_loguniform('b1', 1-1e-1,1-1e-3),0.99))
  lossff = torch.nn.BCELoss()

  total_loss = 0
  weighted_loss = 0
  exp_param = 0.8

  wloss = []

  for i in range(400):
    print("Epoch:", i)
    model.train()
    for data in train_loader:
          data = data.to(device)
          optimizer.zero_grad()
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          loss.backward()
          optimizer.step()
          total_loss += float(loss) * data.num_graphs
    print("Train: ",total_loss / len(train_loader.dataset))

    model.eval()
    total_loss = 0
    for data in val_loader:
          data = data.to(device)
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          total_loss += float(loss) * data.num_graphs
          
    print("Test", total_loss / len(val_loader.dataset))

    weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
    print(weighted_loss/(1-exp_param**(i+1)))
    wloss.append(weighted_loss/(1-exp_param**(i+1)))

    if(i-30>=0 and wloss[i-20]-weighted_loss<0.01):
      break

  return weighted_loss

In [ ]:
# TPE : iterative process that uses history or evaluated hyperparametes to create prob. model which suggest next hyperparameter

In [12]:
import optuna
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(),pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=30)

[I 2023-04-17 19:38:13,088] A new study created in memory with name: no-name-06030083-3fc0-43d3-9a99-3c4a897919a4
<ipython-input-11-543a82f29da4>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  optimizer = torch.optim.Adam(model.parameters(),lr=trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),betas=(trial.suggest_loguniform('b1', 1-1e-1,1-1e-3),0.99))


Epoch: 0
Train:  0.693031710364443
Test 0.6947778860727946
0.6947778860727944
Epoch: 1
Train:  1.0397571995581463
Test 0.6940625399460286
0.6943804715579247
Epoch: 2
Train:  1.0387913008312604
Test 0.6932288877693288
0.6939085109888281
Epoch: 3
Train:  1.037541572100077
Test 0.6921464466786647
0.6933116057347075
Epoch: 4
Train:  1.0358875031436319
Test 0.6907368040346837
0.6925456604407929
Epoch: 5
Train:  1.0336406151235322
Test 0.6889143059978555
0.6915613614873555
Epoch: 6
Train:  1.0305389975890136
Test 0.686090465867039
0.6901768237560675
Epoch: 7
Train:  1.0261699441588406
Test 0.6824093699018597
0.6883101585278992
Epoch: 8
Train:  1.0201969201311523
Test 0.6777031076696766
0.6858598771577992
Epoch: 9
Train:  1.0123780950521812
Test 0.6708581962864915
0.6824986297962468
Epoch: 10
Train:  1.0014727128731025
Test 0.6608651479085287
0.6777653475715466
Epoch: 11
Train:  0.9852253805586707
Test 0.6472040995573386
0.671202073187348
Epoch: 12
Train:  0.96342543280605
Test 0.628127573392

[I 2023-04-17 19:39:10,374] Trial 0 finished with value: 0.13482499771436204 and parameters: {'layer_size1': 512, 'layer_size2': 256, 'layer_size3': 256, 'layer_size4': 192, 'layer_size5': 192, 'learning_rate': 9.138773262152432e-06, 'b1': 0.9650265253690663}. Best is trial 0 with value: 0.13482499771436204.


Train:  0.18604317064577844
Test 0.13646925697205486
0.13482500907334657
Epoch: 0
Train:  0.6903570905709878
Test 0.681168719962403
0.6811687199624031
Epoch: 1
Train:  0.9847739910904741
Test 0.5366530889993185
0.6008822583162451
Epoch: 2
Train:  0.6830578332855588
Test 0.2761192712045851
0.4677826734344172
Epoch: 3
Train:  0.3843477311042639
Test 0.161328371436823
0.36397024050840293
Epoch: 4
Train:  0.2644373882739317
Test 0.14856876129959964
0.2998931703011197
Epoch: 5
Train:  0.25920980015680906
Test 0.1635670108454568
0.2629412015007948
Epoch: 6
Train:  0.24634623663984376
Test 0.14683097937304676
0.23355680181912358
Epoch: 7
Train:  0.22901001370992963
Test 0.14678980636618513
0.2127050611580713
Epoch: 8
Train:  0.22466764925218327
Test 0.1360586561781146
0.19499936136292545
Epoch: 9
Train:  0.2194440903299021
Test 0.14112842813366172
0.18292914511618807
Epoch: 10
Train:  0.20436600161266033
Test 0.14643792295828462
0.17494507421654953
Epoch: 11
Train:  0.20675614176764948
Test 0

[I 2023-04-17 19:39:32,911] Trial 1 finished with value: 0.14163589020703352 and parameters: {'layer_size1': 384, 'layer_size2': 256, 'layer_size3': 256, 'layer_size4': 64, 'layer_size5': 256, 'learning_rate': 8.026461115689728e-05, 'b1': 0.9003918746833378}. Best is trial 0 with value: 0.13482499771436204.


...

0.13612546139284148
0.13663338065981043
Epoch: 0
Train:  0.6944143536326649
Test 0.6927124253122798
0.6927124253122797
Epoch: 1
Train:  1.0405013613648466
Test 0.6925721157601464
0.6926344755610947
Epoch: 2
Train:  1.040121662201899
Test 0.6924509434472947
0.6925592574816685
Epoch: 3
Train:  1.0398548007448079
Test 0.692334417458419
0.6924830921621393
Epoch: 4
Train:  1.0395830454843822
Test 0.692217522488409
0.6924040911882786
Epoch: 5
Train:  1.0393202365973058
Test 0.6921120689902114
0.6923249369364822
Epoch: 6
Train:  1.0390806534351447
Test 0.6920004283988869
0.6922428124827169
Epoch: 7
Train:  1.0387491297809195
Test 0.6918709740970598
0.6921534527262211
Epoch: 8
Train:  1.0384693837864496
Test 0.691724011967907
0.6920542497805839
Epoch: 9
Train:  1.038152928644921
Test 0.6915512591491252
0.6919415506805712
Epoch: 10
Train:  1.0378131089193043
Test 0.6913691093196799
0.6918163037817013
Epoch: 11
Train:  1.037429860769174
Test 0.6911761747611748
0.6916788309140668
Epoch: 12
Train:

[I 2023-04-17 19:55:36,147] Trial 28 finished with value: 0.6825611109237676 and parameters: {'layer_size1': 384, 'layer_size2': 256, 'layer_size3': 384, 'layer_size4': 128, 'layer_size5': 192, 'learning_rate': 2.882858173076284e-06, 'b1': 0.970648239456188}. Best is trial 4 with value: 0.12012299756208797.


Train:  1.019816477875133
Test 0.6796665899046175
0.683237756824187
Epoch: 0
Train:  0.6928410960204435
Test 0.6936368612579373
0.6936368612579373
Epoch: 1
Train:  1.0391838824574327
Test 0.6930863167340066
0.693331003189087
Epoch: 2
Train:  1.0383707282744048
Test 0.6926849006296514
0.6930662070581708
Epoch: 3
Train:  1.0375672736010708
Test 0.6918971647709717
0.6926701900232117
Epoch: 4
Train:  1.03632567238895
Test 0.6908872970294603
0.6921398196657178
Epoch: 5
Train:  1.0347451596033006
Test 0.6894219262259347
0.6914031194315846
Epoch: 6
Train:  1.0325479948476994
Test 0.6878054849394075
0.6904926541177693
Epoch: 7
Train:  1.0298099308223514
Test 0.6855513870497763
0.6893051747207828
Epoch: 8
Train:  1.0260059973259112
Test 0.6821243409272079
0.6876463664901669
Epoch: 9
Train:  1.0206982502133854
Test 0.6778192114044022
0.6854445132853857
Epoch: 10
Train:  1.0140084087193668
Test 0.6725088702890026
0.6826142685112143
Epoch: 11
Train:  1.005816719034216
Test 0.6654796312143515
0.678

[I 2023-04-17 19:56:29,235] Trial 29 finished with value: 0.13358419971399493 and parameters: {'layer_size1': 512, 'layer_size2': 256, 'layer_size3': 256, 'layer_size4': 192, 'layer_size5': 192, 'learning_rate': 8.349149027375222e-06, 'b1': 0.9684160965942011}. Best is trial 4 with value: 0.12012299756208797.


Train:  0.186501651107685
Test 0.13216957160111367
0.13358420160217777


In [13]:
best_trial = study.best_trial

In [14]:
print(best_trial.params)

{'layer_size1': 384, 'layer_size2': 256, 'layer_size3': 384, 'layer_size4': 256, 'layer_size5': 256, 'learning_rate': 1.6864191696897905e-05, 'b1': 0.9652220612376727}


In [15]:
import json

In [16]:
with open('mydata.json', 'w') as f:
    json.dump(best_trial.params, f)

In [17]:
model = Net(768,[384,256,512,64,192],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1.713644852336342e-05,betas=(0.9308401877808863,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []

for i in range(800):
  print("Epoch:", i)
  model.train()
  for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(train_loader.dataset))

  model.eval()
  total_loss = 0
  for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        total_loss += float(loss) * data.num_graphs
        
  print("Test", total_loss / len(val_loader.dataset))

  weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
  print(weighted_loss/(1-exp_param**(i+1)))
  wloss.append(weighted_loss/(1-exp_param**(i+1)))

  if(i-30>=0 and wloss[i-20]-weighted_loss<0.01):
    break

print(weighted_loss)

Epoch: 0
Train:  0.6926371863473466
Test 0.6928813400722685
0.6928813400722684
Epoch: 1
Train:  1.037999260774899
Test 0.6917444578893892
0.6922497388595579
Epoch: 2
Train:  1.0359904295577234
Test 0.6899705507379749
0.6913156453671058
Epoch: 3
Train:  1.031982572414936
Test 0.6862316055175586
0.6895934096457145
Epoch: 4
Train:  1.0243242744342749
Test 0.6784995313965794
0.6862932316801222
Epoch: 5
Train:  1.0086330810745994
Test 0.6616282705422286
0.6796076558664421
Epoch: 6
Train:  0.9758044331720024
Test 0.626516233215402
0.6661716331437375
Epoch: 7
Train:  0.9114698578804841
Test 0.5612173318426251
0.6409491408750069
Epoch: 8
Train:  0.8032648786083683
Test 0.4634798083986555
0.5999528446430713
Epoch: 9
Train:  0.6541563224443149
Test 0.3577538015859904
0.5456861993631307
Epoch: 10
Train:  0.5085341170048102
Test 0.27070469712163064
0.48552181789941623
Epoch: 11
Train:  0.4025209209857843
Test 0.21629371005536874
0.42770291132539207
Epoch: 12
Train:  0.33826767464915475
Test 0.1946

In [18]:
from sklearn.metrics import accuracy_score, f1_score

In [19]:
@torch.no_grad()
def val(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(val_loader.dataset), accuracy, f1

@torch.no_grad()
def test(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [20]:
val(model),test(model)

((0.13787816130238903, 0.9468864468864469, 0.9499136442141622),
 (0.14472380657259218, 0.9566126502875065, 0.9575881451200817))

In [21]:
final = DataLoader(train_data_gos+val_data_gos, batch_size=32, shuffle=True)

In [22]:
model = Net(768,[384,256,512,64,192],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1.713644852336342e-05,betas=(0.9308401877808863,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []

for i in range(70):
  print("Epoch:", i)
  model.train()
  total_loss = 0
  for data in final:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(final.dataset))

Epoch: 0
Train:  0.6928797637149965
Epoch: 1
Train:  0.6903672137539902
Epoch: 2
Train:  0.6847721721808698
Epoch: 3
Train:  0.6707307261861725
Epoch: 4
Train:  0.6355784279959542
Epoch: 5
Train:  0.560017455774058
Epoch: 6
Train:  0.4390554442656055
Epoch: 7
Train:  0.32349471182031364
Epoch: 8
Train:  0.2537925738809455
Epoch: 9
Train:  0.21585180332954695
Epoch: 10
Train:  0.19806776112980312
Epoch: 11
Train:  0.18412879605405716
Epoch: 12
Train:  0.17411073347296616
Epoch: 13
Train:  0.17462209408354556
Epoch: 14
Train:  0.16454003193621525
Epoch: 15
Train:  0.16357310190887825
Epoch: 16
Train:  0.15783987660984417
Epoch: 17
Train:  0.15851857232086825
Epoch: 18
Train:  0.151304594617478
Epoch: 19
Train:  0.1543248496422037
Epoch: 20
Train:  0.14625036869293603
Epoch: 21
Train:  0.1465473059378337
Epoch: 22
Train:  0.14274161737480442
Epoch: 23
Train:  0.13940603605793364
Epoch: 24
Train:  0.13774213669719276
Epoch: 25
Train:  0.13473597507337073
Epoch: 26
Train:  0.130775961501839